The notebook is for pipeline all data preparation of the football match data

In [5]:
import re
import pandas as pd
import os
import numpy as np
from csv import reader
import plotly.express as px
import missingno as msno
import pickle

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

pd.options.mode.chained_assignment = None

In [6]:
# define recent matches meaning
RECENT_PREFORMANCE_MATCH_COUNT = 3

In [7]:
# load pickle and read content
d = pickle.load(open('./ELO/elo_dict.pkl', 'rb'))

Functions to get total goals so far for home team and away team in each game. The dataframe will call the apply() so that it will loop all records in the dataframe. And it will filter all records which round < current round and seperated with home and away for each team. The filter records then can calculate home total goals and away total goals

In [8]:
def getLeagueSeasonTeamBeforeRoundTotalGoal(data, league, season, team, round):
    # determine home or away and get the score 
    # get home game of the team
    home_pd = data[(data["League"]==league) & (data["Home_Team"]==team) & (data["Season"]==season) & (data["Round"]<round)]
    df_home_score_sofar =  home_pd['Result'].str.extract(r'(\d)-\d')
    home_total_score = df_home_score_sofar[0].astype('Int64').sum()

    # get away game of the team
    away_pd = data[(data["League"]==league) & (data["Away_Team"]==team) & (data["Season"]==season) & (data["Round"]<round)]
    df_away_score_sofar =  away_pd['Result'].str.extract(r'\d-(\d)')
    away_total_score = df_away_score_sofar[0].astype('Int64').sum()

    # calculate total goals
    return (home_total_score + away_total_score)


def fillWithTotalGoalSoFar(record, data):
    # get home team and away team and round
    league = record['League']
    season = record['Season']
    round = record['Round']
    hteam = record['Home_Team']
    ateam = record['Away_Team']
    
    home_goal_so_far = getLeagueSeasonTeamBeforeRoundTotalGoal(data, league, season, hteam, round)
    away_goal_so_far = getLeagueSeasonTeamBeforeRoundTotalGoal(data, league, season, ateam, round)

    return [home_goal_so_far, away_goal_so_far]

Function to get elo home and away for each record in the dataframe by apply()

In [9]:
def fillWithELO(link):
    if link not in d:
        return [pd.NA, pd.NA]
    else:
        return [d[link]['Elo_home'], d[link]['Elo_away']]

Function to get recent performance with apply() similarly

In [10]:
def findRecentPreviousRounds(currentRound, limit):
    if currentRound<=limit:
        return None
    else:
        r = []
        for l in range(limit):
            r.append(currentRound - (limit-l))
        return r


def findLeagueSeasonTeamRecentPreviousRounds(data, league, season, team, round):
    rounds = findRecentPreviousRounds(round, RECENT_PREFORMANCE_MATCH_COUNT)         # by definition is 6, can change for optimization
    if rounds is None:
        return None

    previous_matches_pd =  data[(data["League"]==league) & ((data["Home_Team"]==team) | (data["Away_Team"]==team)) & (data["Season"]==season) & (data["Round"].isin(rounds))]
    recent_perf = 0
    for index, row in previous_matches_pd.iterrows():
        hteam = row['Home_Team']
        ateam = row['Away_Team']
        if hteam==team:
            recent_perf = recent_perf + (row['Home_Score']-row['Away_Score'])
        else:
            recent_perf = recent_perf + (row['Away_Score']-row['Home_Score'])

    return recent_perf


def fillWithRecentPerformance(record, data):
    # get home team and away team and round
    league = record['League']
    season = record['Season']
    round = record['Round']
    hteam = record['Home_Team']
    ateam = record['Away_Team']
    
    home_team_goal_diff = findLeagueSeasonTeamRecentPreviousRounds(data, league, season, hteam, round)
    away_team_goal_diff = findLeagueSeasonTeamRecentPreviousRounds(data, league, season, ateam, round)

    return [home_team_goal_diff, away_team_goal_diff]

In [11]:
# load all directory as league name list
dir = "./Results"
leagues = [name for name in os.listdir(dir) if os.path.isdir(os.path.join(dir, name))]

# loop to open csv
result_with_goal_sofar_pd = pd.DataFrame()
for league in leagues:
    print("process league: " + league + "...")
    league_folder = os.path.join(dir, league)
    csv_file_for_league = [os.path.join(league_folder, name) for name in os.listdir(league_folder) if name.endswith('.csv')]
    
    for csv_filename in csv_file_for_league:
        current_league_season_pd = pd.read_csv(csv_filename, skiprows=[0], names=["Home_Team", "Away_Team", "Result", "Link", "Season", "Round", "League"])

        # Divide result into home_score and away_score
        df_score =  current_league_season_pd['Result'].str.extract(r'(\d)-(\d)')
        current_league_season_pd.insert(loc=3, column="Home_Score", value=df_score[0].astype('Int64'))     # use Int64 as it support NaN
        current_league_season_pd.insert(loc=4, column="Away_Score", value=df_score[1].astype('Int64')) 

        if len(current_league_season_pd)>0:
            # get home team and away team total goal so far
            home_away_total_goal_sofar = current_league_season_pd.apply(fillWithTotalGoalSoFar, data=current_league_season_pd, axis=1)
            goal_so_far_list = np.array(home_away_total_goal_sofar.values.tolist())         # convert to list
            home_away_total_goal_sofar_pd = pd.DataFrame(goal_so_far_list, columns=["HOME_GOAL_SO_FAR", "AWAY_GOAL_SO_FAR"])    # convert to dataframe
            current_league_season_pd.insert(loc=5, column="HOME_TOTAL_GOAL_SO_FAR", value=home_away_total_goal_sofar_pd["HOME_GOAL_SO_FAR"].astype('Int64')) 
            current_league_season_pd.insert(loc=6, column="AWAY_TOTAL_GOAL_SO_FAR", value=home_away_total_goal_sofar_pd["AWAY_GOAL_SO_FAR"].astype('Int64'))     

            # merge with ELO
            result_elo_pd = current_league_season_pd['Link'].apply(fillWithELO)   
            elo_list = np.array(result_elo_pd.values.tolist())
            elo_df = pd.DataFrame(elo_list, columns=["ELO_HOME", "ELO_AWAY"])
            current_league_season_pd.insert(loc=8, column="ELO_HOME", value=elo_df["ELO_HOME"].astype('Int64')) 
            current_league_season_pd.insert(loc=9, column="ELO_AWAY", value=elo_df["ELO_AWAY"].astype('Int64')) 

            # get recent performance
            home_away_recent_perf = current_league_season_pd.apply(fillWithRecentPerformance, data=current_league_season_pd, axis=1)
            perf_list = np.array(home_away_recent_perf.values.tolist())
            home_away_perf_pd = pd.DataFrame(perf_list, columns=["HOME_LASTEST_GOAL_DIFF", "AWAY_LASTEST_GOAL_DIFF"])
            current_league_season_pd.insert(loc=7, column="HOME_LASTEST_GOAL_DIFF", value=home_away_perf_pd["HOME_LASTEST_GOAL_DIFF"].astype('Int64')) 
            current_league_season_pd.insert(loc=8, column="AWAY_LASTEST_GOAL_DIFF", value=home_away_perf_pd["AWAY_LASTEST_GOAL_DIFF"].astype('Int64')) 

            result_with_goal_sofar_pd = pd.concat([result_with_goal_sofar_pd, current_league_season_pd])

# export to csv
result_with_goal_sofar_pd.to_csv('cleaned_dataset.csv', index=False)

process league: championship...
process league: eerste_divisie...
process league: primeira_liga...
process league: ligue_1...
process league: segunda_division...
process league: 2_liga...
process league: serie_a...
process league: bundesliga...
process league: primera_division...
process league: ligue_2...
process league: premier_league...
process league: eredivisie...
process league: segunda_liga...
process league: serie_b...


In [12]:
result_with_goal_sofar_pd

,Home_Team,Away_Team,Result,Home_Score,Away_Score,HOME_TOTAL_GOAL_SO_FAR,AWAY_TOTAL_GOAL_SO_FAR,HOME_LASTEST_GOAL_DIFF,AWAY_LASTEST_GOAL_DIFF,Link,ELO_HOME,ELO_AWAY,Season,Round,League
0,Watford,Middlesbrough,1-0,1,0,0,0,<NA>,<NA>,https://www.besoccer.com/match/watford-fc/midd...,65,60,2021,1,championship
1,Birmingham City,Brentford,1-0,1,0,0,0,<NA>,<NA>,https://www.besoccer.com/match/birmingham-city...,52,59,2021,1,championship
2,Wycombe Wanderers,Rotherham United,0-1,0,1,0,0,<NA>,<NA>,https://www.besoccer.com/match/wycombe-wandere...,41,48,2021,1,championship
3,AFC Bournemouth,Blackburn Rovers,3-2,3,2,0,0,<NA>,<NA>,https://www.besoccer.com/match/afc-bournemouth...,63,57,2021,1,championship
4,Barnsley,Luton Town,0-1,0,1,0,0,<NA>,<NA>,https://www.besoccer.com/match/barnsley-fc/lut...,47,50,2021,1,championship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Pescara,Padova,1-2,1,2,49,39,5,-3,https://www.besoccer.com/match/pescara-calcio/...,59,54,1997,38,serie_b
376,Genoa,Palermo FC,4-1,4,1,54,39,2,-1,https://www.besoccer.com/match/genoa/palermo/1...,61,58,1997,38,serie_b
377,Torino,Ravenna FC,0-4,0,4,45,39,-2,-2,https://www.besoccer.com/match/torino-fc/raven...,63,54,1997,38,serie_b
378,Salernitana,Reggina,1-3,1,3,30,37,-2,3,https://www.besoccer.com/match/salernitana-cal...,52,52,1997,38,serie_b


In [8]:
# delete no value column
result_with_goal_sofar_pd.drop('Result', inplace=True, axis=1)
result_with_goal_sofar_pd.drop('Link', inplace=True, axis=1)

In [9]:
# reorder dataframe column
result_with_goal_sofar_pd.insert(0, 'League', result_with_goal_sofar_pd.pop('League'))
result_with_goal_sofar_pd.insert(1, 'Season', result_with_goal_sofar_pd.pop('Season'))
result_with_goal_sofar_pd.insert(2, 'Round', result_with_goal_sofar_pd.pop('Round'))
result_with_goal_sofar_pd.insert(5, 'ELO_HOME', result_with_goal_sofar_pd.pop('ELO_HOME'))
result_with_goal_sofar_pd.insert(6, 'ELO_AWAY', result_with_goal_sofar_pd.pop('ELO_AWAY'))

In [10]:
result_with_goal_sofar_pd.head(30)

,League,Season,Round,Home_Team,Away_Team,ELO_HOME,ELO_AWAY,Home_Score,Away_Score,HOME_TOTAL_GOAL_SO_FAR,AWAY_TOTAL_GOAL_SO_FAR,HOME_LASTEST_GOAL_DIFF,AWAY_LASTEST_GOAL_DIFF
0,championship,2021,1,Watford,Middlesbrough,65,60,1,0,0,0,<NA>,<NA>
1,championship,2021,1,Birmingham City,Brentford,52,59,1,0,0,0,<NA>,<NA>
2,championship,2021,1,Wycombe Wanderers,Rotherham United,41,48,0,1,0,0,<NA>,<NA>
3,championship,2021,1,AFC Bournemouth,Blackburn Rovers,63,57,3,2,0,0,<NA>,<NA>
4,championship,2021,1,Barnsley,Luton Town,47,50,0,1,0,0,<NA>,<NA>
5,championship,2021,1,Bristol City,Coventry City,58,45,2,1,0,0,<NA>,<NA>
6,championship,2021,1,Cardiff City,Sheffield Wednesday,60,60,0,2,0,0,<NA>,<NA>
7,championship,2021,1,Derby County,Reading,61,54,0,2,0,0,<NA>,<NA>
8,championship,2021,1,Huddersfield Town,Norwich City,57,61,0,1,0,0,<NA>,<NA>
9,championship,2021,1,Millwall,Stoke City,55,65,0,0,0,0,<NA>,<NA>


In [11]:
result_with_goal_sofar_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146641 entries, 0 to 379
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   League                  146641 non-null  object
 1   Season                  146641 non-null  int64 
 2   Round                   146641 non-null  int64 
 3   Home_Team               146641 non-null  object
 4   Away_Team               146641 non-null  object
 5   ELO_HOME                122549 non-null  Int64 
 6   ELO_AWAY                122549 non-null  Int64 
 7   Home_Score              146545 non-null  Int64 
 8   Away_Score              146545 non-null  Int64 
 9   HOME_TOTAL_GOAL_SO_FAR  146641 non-null  Int64 
 10  AWAY_TOTAL_GOAL_SO_FAR  146641 non-null  Int64 
 11  HOME_LASTEST_GOAL_DIFF  133847 non-null  Int64 
 12  AWAY_LASTEST_GOAL_DIFF  133843 non-null  Int64 
dtypes: Int64(8), int64(2), object(3)
memory usage: 16.8+ MB


In [229]:
# load csv (reset data)
result_with_goal_sofar_pd = pd.read_csv('cleaned_dataset.csv')
result_with_goal_sofar_pd

,Home_Team,Away_Team,Result,Home_Score,Away_Score,HOME_TOTAL_GOAL_SO_FAR,AWAY_TOTAL_GOAL_SO_FAR,HOME_LASTEST_GOAL_DIFF,AWAY_LASTEST_GOAL_DIFF,Link,ELO_HOME,ELO_AWAY,Season,Round,League
0,Watford,Middlesbrough,1-0,1.0,0.0,0,0,NaN,NaN,https://www.besoccer.com/match/watford-fc/midd...,65.0,60.0,2021,1,championship
1,Birmingham City,Brentford,1-0,1.0,0.0,0,0,NaN,NaN,https://www.besoccer.com/match/birmingham-city...,52.0,59.0,2021,1,championship
2,Wycombe Wanderers,Rotherham United,0-1,0.0,1.0,0,0,NaN,NaN,https://www.besoccer.com/match/wycombe-wandere...,41.0,48.0,2021,1,championship
3,AFC Bournemouth,Blackburn Rovers,3-2,3.0,2.0,0,0,NaN,NaN,https://www.besoccer.com/match/afc-bournemouth...,63.0,57.0,2021,1,championship
4,Barnsley,Luton Town,0-1,0.0,1.0,0,0,NaN,NaN,https://www.besoccer.com/match/barnsley-fc/lut...,47.0,50.0,2021,1,championship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146636,Pescara,Padova,1-2,1.0,2.0,49,39,5.0,-3.0,https://www.besoccer.com/match/pescara-calcio/...,59.0,54.0,1997,38,serie_b
146637,Genoa,Palermo FC,4-1,4.0,1.0,54,39,2.0,-1.0,https://www.besoccer.com/match/genoa/palermo/1...,61.0,58.0,1997,38,serie_b
146638,Torino,Ravenna FC,0-4,0.0,4.0,45,39,-2.0,-2.0,https://www.besoccer.com/match/torino-fc/raven...,63.0,54.0,1997,38,serie_b
146639,Salernitana,Reggina,1-3,1.0,3.0,30,37,-2.0,3.0,https://www.besoccer.com/match/salernitana-cal...,52.0,52.0,1997,38,serie_b


In [230]:
# clean up records with NaN
# drop record with na
result_with_goal_sofar_pd = result_with_goal_sofar_pd.dropna()
result_with_goal_sofar_pd

,Home_Team,Away_Team,Result,Home_Score,Away_Score,HOME_TOTAL_GOAL_SO_FAR,AWAY_TOTAL_GOAL_SO_FAR,HOME_LASTEST_GOAL_DIFF,AWAY_LASTEST_GOAL_DIFF,Link,ELO_HOME,ELO_AWAY,Season,Round,League
36,Coventry City,AFC Bournemouth,1-3,1.0,3.0,4,5,0.0,2.0,https://www.besoccer.com/match/coventry-city/a...,46.0,62.0,2021,4,championship
37,Norwich City,Derby County,0-1,0.0,1.0,3,1,0.0,-7.0,https://www.besoccer.com/match/norwich-city-fc...,62.0,60.0,2021,4,championship
38,Blackburn Rovers,Cardiff City,0-0,0.0,0.0,11,3,8.0,-1.0,https://www.besoccer.com/match/blackburn-rover...,58.0,60.0,2021,4,championship
39,Luton Town,Wycombe Wanderers,2-0,2.0,0.0,3,0,1.0,-8.0,https://www.besoccer.com/match/luton-town-fc/w...,51.0,41.0,2021,4,championship
40,Middlesbrough,Barnsley,2-1,2.0,1.0,2,0,-1.0,-3.0,https://www.besoccer.com/match/middlesbrough-f...,61.0,46.0,2021,4,championship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146636,Pescara,Padova,1-2,1.0,2.0,49,39,5.0,-3.0,https://www.besoccer.com/match/pescara-calcio/...,59.0,54.0,1997,38,serie_b
146637,Genoa,Palermo FC,4-1,4.0,1.0,54,39,2.0,-1.0,https://www.besoccer.com/match/genoa/palermo/1...,61.0,58.0,1997,38,serie_b
146638,Torino,Ravenna FC,0-4,0.0,4.0,45,39,-2.0,-2.0,https://www.besoccer.com/match/torino-fc/raven...,63.0,54.0,1997,38,serie_b
146639,Salernitana,Reggina,1-3,1.0,3.0,30,37,-2.0,3.0,https://www.besoccer.com/match/salernitana-cal...,52.0,52.0,1997,38,serie_b


In [231]:

# delete no value column
result_with_goal_sofar_pd.drop('Result', inplace=True, axis=1)
result_with_goal_sofar_pd.drop('Link', inplace=True, axis=1)
#result_with_goal_sofar_pd.drop('League', inplace=True, axis=1)
#result_with_goal_sofar_pd.drop('Season', inplace=True, axis=1)
#result_with_goal_sofar_pd.drop('Round', inplace=True, axis=1)
#result_with_goal_sofar_pd.drop('Home_Team', inplace=True, axis=1)
#result_with_goal_sofar_pd.drop('Away_Team', inplace=True, axis=1)
result_with_goal_sofar_pd

,Home_Team,Away_Team,Home_Score,Away_Score,HOME_TOTAL_GOAL_SO_FAR,AWAY_TOTAL_GOAL_SO_FAR,HOME_LASTEST_GOAL_DIFF,AWAY_LASTEST_GOAL_DIFF,ELO_HOME,ELO_AWAY,Season,Round,League
36,Coventry City,AFC Bournemouth,1.0,3.0,4,5,0.0,2.0,46.0,62.0,2021,4,championship
37,Norwich City,Derby County,0.0,1.0,3,1,0.0,-7.0,62.0,60.0,2021,4,championship
38,Blackburn Rovers,Cardiff City,0.0,0.0,11,3,8.0,-1.0,58.0,60.0,2021,4,championship
39,Luton Town,Wycombe Wanderers,2.0,0.0,3,0,1.0,-8.0,51.0,41.0,2021,4,championship
40,Middlesbrough,Barnsley,2.0,1.0,2,0,-1.0,-3.0,61.0,46.0,2021,4,championship
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146636,Pescara,Padova,1.0,2.0,49,39,5.0,-3.0,59.0,54.0,1997,38,serie_b
146637,Genoa,Palermo FC,4.0,1.0,54,39,2.0,-1.0,61.0,58.0,1997,38,serie_b
146638,Torino,Ravenna FC,0.0,4.0,45,39,-2.0,-2.0,63.0,54.0,1997,38,serie_b
146639,Salernitana,Reggina,1.0,3.0,30,37,-2.0,3.0,52.0,52.0,1997,38,serie_b


In [232]:

# reorder dataframe column
result_with_goal_sofar_pd.insert(0, 'League', result_with_goal_sofar_pd.pop('League'))
result_with_goal_sofar_pd.insert(1, 'Season', result_with_goal_sofar_pd.pop('Season'))
result_with_goal_sofar_pd.insert(2, 'Round', result_with_goal_sofar_pd.pop('Round'))
result_with_goal_sofar_pd.insert(5, 'ELO_HOME', result_with_goal_sofar_pd.pop('ELO_HOME'))
result_with_goal_sofar_pd.insert(6, 'ELO_AWAY', result_with_goal_sofar_pd.pop('ELO_AWAY'))
#result_with_goal_sofar_pd.insert(0, 'ELO_HOME', result_with_goal_sofar_pd.pop('ELO_HOME'))
#result_with_goal_sofar_pd.insert(1, 'ELO_AWAY', result_with_goal_sofar_pd.pop('ELO_AWAY'))
result_with_goal_sofar_pd

,League,Season,Round,Home_Team,Away_Team,ELO_HOME,ELO_AWAY,Home_Score,Away_Score,HOME_TOTAL_GOAL_SO_FAR,AWAY_TOTAL_GOAL_SO_FAR,HOME_LASTEST_GOAL_DIFF,AWAY_LASTEST_GOAL_DIFF
36,championship,2021,4,Coventry City,AFC Bournemouth,46.0,62.0,1.0,3.0,4,5,0.0,2.0
37,championship,2021,4,Norwich City,Derby County,62.0,60.0,0.0,1.0,3,1,0.0,-7.0
38,championship,2021,4,Blackburn Rovers,Cardiff City,58.0,60.0,0.0,0.0,11,3,8.0,-1.0
39,championship,2021,4,Luton Town,Wycombe Wanderers,51.0,41.0,2.0,0.0,3,0,1.0,-8.0
40,championship,2021,4,Middlesbrough,Barnsley,61.0,46.0,2.0,1.0,2,0,-1.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146636,serie_b,1997,38,Pescara,Padova,59.0,54.0,1.0,2.0,49,39,5.0,-3.0
146637,serie_b,1997,38,Genoa,Palermo FC,61.0,58.0,4.0,1.0,54,39,2.0,-1.0
146638,serie_b,1997,38,Torino,Ravenna FC,63.0,54.0,0.0,4.0,45,39,-2.0,-2.0
146639,serie_b,1997,38,Salernitana,Reggina,52.0,52.0,1.0,3.0,30,37,-2.0,3.0


In [233]:
# find who win H:Home A:Away D:Draw
def get_result(record):
    hscore = record['Home_Score']
    ascore = record['Away_Score']
    if hscore is pd.NA or ascore is pd.NA:
        return pd.NA
    if hscore==ascore:
        return 0
    elif hscore>ascore:
        return 1
    else:
        return 2

result_pd = result_with_goal_sofar_pd.apply(get_result, axis=1)

#result_with_goal_sofar_pd.drop('Home_Score', inplace=True, axis=1)
#result_with_goal_sofar_pd.drop('Away_Score', inplace=True, axis=1)

result_with_goal_sofar_pd.insert(loc=len(result_with_goal_sofar_pd.columns), column="Result", value=result_pd.astype('Int64')) 
result_with_goal_sofar_pd

,League,Season,Round,Home_Team,Away_Team,ELO_HOME,ELO_AWAY,Home_Score,Away_Score,HOME_TOTAL_GOAL_SO_FAR,AWAY_TOTAL_GOAL_SO_FAR,HOME_LASTEST_GOAL_DIFF,AWAY_LASTEST_GOAL_DIFF,Result
36,championship,2021,4,Coventry City,AFC Bournemouth,46.0,62.0,1.0,3.0,4,5,0.0,2.0,2
37,championship,2021,4,Norwich City,Derby County,62.0,60.0,0.0,1.0,3,1,0.0,-7.0,2
38,championship,2021,4,Blackburn Rovers,Cardiff City,58.0,60.0,0.0,0.0,11,3,8.0,-1.0,0
39,championship,2021,4,Luton Town,Wycombe Wanderers,51.0,41.0,2.0,0.0,3,0,1.0,-8.0,1
40,championship,2021,4,Middlesbrough,Barnsley,61.0,46.0,2.0,1.0,2,0,-1.0,-3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146636,serie_b,1997,38,Pescara,Padova,59.0,54.0,1.0,2.0,49,39,5.0,-3.0,2
146637,serie_b,1997,38,Genoa,Palermo FC,61.0,58.0,4.0,1.0,54,39,2.0,-1.0,1
146638,serie_b,1997,38,Torino,Ravenna FC,63.0,54.0,0.0,4.0,45,39,-2.0,-2.0,2
146639,serie_b,1997,38,Salernitana,Reggina,52.0,52.0,1.0,3.0,30,37,-2.0,3.0,2
